# 06_.NuSVC.ipynb

We will use the train/test data generated from *00_create_dataset.ipynb* and perform basic **MODEL_TYPE**

In [1]:
''' data and math '''
import pandas as pd
import numpy as np

''' plotting images '''
from matplotlib import pyplot as plt
%matplotlib inline

''' traversing directories '''
import os
from pathlib import Path

''' utilities '''
from tqdm import tqdm


In [2]:
''' used to reference the root directory, for directory traversal ''' 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
mount_dir = '/content/gdrive'
root_dir = Path('/content/gdrive/My Drive/it3011_project')

Mounted at /content/gdrive


# Helper functions

In [3]:
# create the utility score, which takes in the prediction value and the ground truth action and generates a score
# link: https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation

# data: original train/test data    action: the y-value. can either be y_pred or original values too, if we want the max score attainable
def utility_score(data, action): 
  dates_set = set(data.date.values)
  dates = data.loc[:, ['date']].values.flatten()
  weights = data.loc[:, ['weight']].values.flatten()
  resps = data.loc[:, ['resp']].values.flatten()
  actions = action.flatten()

  i = len(dates_set)
  p_i = []

  for date in dates_set:
    indices = np.where(dates == date)[0]
    p_i_temp = 0
    for j in indices:
      p_i_temp = p_i_temp + weights[j] * resps[j] * actions[j]
    p_i.append(p_i_temp)
  
  p_i_squared = [p_i1*p_i2 for p_i1,p_i2 in zip(p_i,p_i)]
  t = ( sum(p_i) / np.sqrt(sum(p_i_squared)) ) * np.sqrt(250/i)
  u = min(max(t, 0), 6) * sum(p_i)

  return u

def max_train_utility_score():
  # value obtained from notebook 01_dataset_understanding.ipynb
  max_achievable_train_utility = 38666.152212179244
  return max_achievable_train_utility

def max_test_utility_score():
  # value obtained from notebook 01_dataset_understanding.ipynb
  max_achievable_test_utility = 15405.02761054398
  return max_achievable_test_utility

# Loading data

In [4]:
# load data
train = pd.read_csv(root_dir/"data/train.csv")
test = pd.read_csv(root_dir/"data/test.csv")
print("data loaded")

data loaded


In [5]:
# check shape
print(train.shape)
print(test.shape)

(280145, 139)
(120504, 139)


In [6]:
# create train/test sets
features = [feature for feature in test.keys() if "feature" in feature]
x_train = train.loc[:, features].values
y_train = train.loc[:,['action']].values
x_test = test.loc[:, features].values
y_test = test.loc[:,['action']].values
print("train/test set created")

train/test set created


# Model

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC

nu_svc = make_pipeline(StandardScaler(), NuSVC())
model = nu_svc.fit(x_train, y_train.ravel())

preds = model.predict(x_test)
print("Number of mislabeled points out of a total %d points : %d" % (x_test.shape[0], (y_test.ravel() != preds).sum()))
print("Classification accuracy: {}".format((preds == y_test.ravel()).mean()))
print(utility_score(y_test, preds))

Not sure why this doesn't seem to be able to run

OK, what even. I'm done here.